In [1]:
!pip install numpy matplotlib tensorflow

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

c:\Users\mural\OneDrive\Desktop\classification_proj\.venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [ ]:
pip install kaggle


In [ ]:

!kaggle datasets download -d anandkumarsahu09/cattle-breeds-dataset

In [7]:
!tar -xf cattle-breeds-dataset.zip


In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,          # normalize pixel values
    rotation_range=10,       # rotate images randomly
    width_shift_range=0.05,   # shift horizontally
    height_shift_range=0.05,  # shift vertically
    zoom_range=0.1,          # random zoomS
    horizontal_flip=True,    # flip horizontally
    validation_split=0.2     # 20% data for validation
)

# Training data
train_data = datagen.flow_from_directory(
    'cattle Breeds/',               # path to dataset folder
    target_size=(224,224),   # resize images
    batch_size=32,
    class_mode='categorical', # multi-class classification
    subset='training'
)

# Validation data
val_data = datagen.flow_from_directory(
    'cattle Breeds/',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 969 images belonging to 5 classes.
Found 239 images belonging to 5 classes.


In [21]:
print("Training samples:", train_data.samples)
print("Validation samples:", val_data.samples)
num_classes = train_data.num_classes
print("Number of breeds:", num_classes)

Training samples: 969
Validation samples: 239
Number of breeds: 5


In [22]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False


In [23]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(train_data.num_classes, activation='softmax')
])


In [24]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [26]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(           #If validation loss does not improve for 5 consecutive epochs, stop training automatically
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(       #If validation loss stops improving, reduce the learning rate.”
        monitor="val_loss",
        factor=0.2,
        patience=3,
        min_lr=1e-6
    )
]

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=30,
    callbacks=callbacks
)


Epoch 1/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 20s 525ms/step - accuracy: 0.2921 - loss: 1.7641 - val_accuracy: 0.4393 - val_loss: 1.3718 - learning_rate: 1.0000e-04
Epoch 2/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 14s 443ms/step - accuracy: 0.4159 - loss: 1.3900 - val_accuracy: 0.5690 - val_loss: 1.1828 - learning_rate: 1.0000e-04
Epoch 3/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 13s 429ms/step - accuracy: 0.5304 - loss: 1.1999 - val_accuracy: 0.6653 - val_loss: 1.0766 - learning_rate: 1.0000e-04
Epoch 4/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 15s 479ms/step - accuracy: 0.5996 - loss: 1.0540 - val_accuracy: 0.6025 - val_loss: 1.0539 - learning_rate: 1.0000e-04
Epoch 5/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 14s 446ms/step - accuracy: 0.6336 - loss: 0.9769 - val_accuracy: 0.6276 - val_loss: 1.0136 - learning_rate: 1.0000e-04
Epoch 6/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 14s 444ms/step - accuracy: 0.6873 - loss: 0.8993 - val_accuracy: 0.6234 - val_loss: 0.9744 - learning_rate: 1.0000e-04
Epoch 7/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 14s 463ms/step - acc

In [14]:
y_true = val_data.classes
y_true


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
      dtype=int32)

In [11]:
y_pred_probs = model.predict(val_data)
y_pred_probs


8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 210ms/step


array([[0.2734715 , 0.27904135, 0.02037651, 0.29676902, 0.13034157],
       [0.2656938 , 0.28282285, 0.08839063, 0.22003932, 0.14305341],
       [0.31985006, 0.19583678, 0.00344693, 0.1847829 , 0.2960833 ],
       ...,
       [0.39406273, 0.332488  , 0.05398371, 0.13412659, 0.08533893],
       [0.08258816, 0.02567778, 0.87768936, 0.01005084, 0.00399392],
       [0.24210572, 0.2117817 , 0.0057464 , 0.20030466, 0.3400615 ]],
      shape=(239, 5), dtype=float32)

In [13]:
import numpy as np

y_pred = np.argmax(y_pred_probs, axis=1)
y_pred


array([3, 1, 0, 4, 4, 1, 2, 1, 4, 2, 4, 2, 4, 2, 2, 0, 4, 0, 4, 0, 0, 4,
       0, 2, 4, 4, 1, 2, 2, 0, 0, 1, 4, 4, 0, 2, 2, 2, 4, 2, 4, 2, 0, 1,
       1, 4, 2, 2, 4, 2, 4, 0, 1, 1, 4, 0, 3, 2, 1, 4, 1, 2, 2, 1, 1, 1,
       4, 3, 3, 2, 1, 0, 4, 1, 2, 1, 3, 4, 2, 1, 4, 2, 1, 2, 2, 2, 0, 4,
       1, 4, 3, 4, 2, 1, 1, 2, 0, 3, 0, 1, 4, 1, 1, 0, 0, 1, 1, 1, 0, 4,
       3, 2, 2, 2, 1, 4, 0, 4, 2, 0, 1, 0, 3, 1, 4, 4, 1, 2, 2, 4, 3, 0,
       2, 4, 4, 1, 4, 1, 2, 2, 4, 3, 4, 0, 2, 1, 4, 4, 2, 1, 4, 0, 2, 4,
       0, 2, 2, 1, 2, 4, 2, 1, 2, 0, 4, 2, 1, 2, 2, 1, 1, 4, 3, 0, 1, 2,
       1, 2, 1, 2, 4, 2, 2, 1, 1, 1, 1, 1, 2, 0, 4, 2, 4, 1, 3, 4, 0, 4,
       0, 2, 4, 1, 4, 1, 4, 4, 4, 4, 4, 1, 4, 0, 2, 2, 1, 0, 4, 2, 2, 4,
       0, 2, 2, 2, 4, 1, 0, 2, 1, 2, 1, 1, 2, 2, 1, 0, 0, 2, 4])

In [16]:
!pip install scikit-learn
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
print(cm)


  Using cached scikit_learn-1.8.0-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.8.0-cp312-cp312-win_amd64.whl (8.0 MB)
Using cached joblib-1.5.3-py3-none-any.whl (309 kB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)

   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   -------------------------- ------------- 2/3 [scikit-learn]
   -------------------------- ------------- 2/3 [scikit-learn]
   -------------------------- -

In [17]:
print(val_data.class_indices)


{'Ayrshire cattle': 0, 'Brown Swiss cattle': 1, 'Holstein Friesian cattle': 2, 'Jersey cattle': 3, 'Red Dane cattle': 4}


In [18]:
from sklearn.metrics import classification_report

print(classification_report(
    y_true,
    y_pred,
    target_names=list(val_data.class_indices.keys())
))


                          precision    recall  f1-score   support

         Ayrshire cattle       0.30      0.21      0.25        52
      Brown Swiss cattle       0.25      0.32      0.28        47
Holstein Friesian cattle       0.16      0.22      0.19        50
           Jersey cattle       0.15      0.04      0.06        50
         Red Dane cattle       0.19      0.30      0.24        40

                accuracy                           0.21       239
               macro avg       0.21      0.22      0.20       239
            weighted avg       0.21      0.21      0.20       239

